In [1]:
import os
import argparse
import torch
import pandas as pd
import csv
import cv2
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from torchvision import models
import torchmetrics

In [93]:
DATA_PATH = "/kaggle/input/vk-made-sports-image-classification"
TRAIN_PATH = DATA_PATH +  "/train"
TEST_PATH  = DATA_PATH + "/test"
TRAIN_LABELS_PATH = DATA_PATH + '/' + "train.csv"
TEST_LABELS_PATH  =  DATA_PATH + '/' + "test.csv"
SPLIT_FRAC = 0.2
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
IMG_SHAPE = ()

In [3]:
sport_enc = LabelEncoder()
labels_file = open(TRAIN_LABELS_PATH, "r")
labels = pd.read_csv(labels_file)
labels['label'] = sport_enc.fit_transform(labels['label'])
print(labels)


                                        image_id  label
0      46514481-2d8b-4d49-8991-012e1bfd34f6.jpeg     24
1      ec66e513-adac-4a30-b6a9-3d647ee6e46b.jpeg     10
2      4d60732e-d680-4bfd-9067-70ff8137f537.jpeg     18
3      93327011-8e3d-4f0d-849d-a26ddaf6488b.jpeg      8
4      b6853478-48c1-48b2-b104-74903730c831.jpeg     19
...                                          ...    ...
45368  b53fcada-9046-44e4-92f9-8c1a0dc60fb3.jpeg     12
45369  53dd096c-3697-4e3b-99b6-e77c1129b798.jpeg     10
45370  5e26219c-4629-42fe-b037-fb6b1bf78f7f.jpeg     23
45371  c43f5525-44e4-48f6-a8ef-95ebe569bfc6.jpeg     16
45372  8fdfbb87-c2ca-4a41-b19c-1799f8bc032c.jpeg      7

[45373 rows x 2 columns]


In [4]:
samples_size = labels['label'].value_counts()
class_weights = 1 - samples_size / len(labels['label'])

In [5]:
N_CLASSES = len(sport_enc.classes_)

In [76]:
class Imagedata(Dataset):
    def load_image(self, path):
        img = Image.open(path).convert('RGB')
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=15),    
        transforms.ToTensor()
        ])
        transform1 = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor() 
        ])
        if self.testing == True:
            res = transform1(img)
        else:
            res = transform(img)
        res.to(DEVICE)
        return  res

    def get_label(self, filename):
        return labels.loc[labels['image_id'] == filename, 'label'].values[0]
    
    def __init__(self, labels_path, data_path, testing = False):
        self.samples = []
        self.testing = testing
        self.symbols = set()
        self.img_shape = None
        with open(labels_path, "r") as labels_file:
            self.labels = pd.read_csv(labels_file)
            if self.testing == False:
                self.labels['label'] = sport_enc.transform(self.labels['label'])
        super(self.__class__, self).__init__()
        for filename in os.listdir(data_path):
            file_path = data_path + "/" + filename
           # print(f"processing {file_path}")
            image = file_path
            if self.testing == True:
                self.samples.append(image)
            else:
                image_label = self.get_label(filename)

                self.samples.append((image, image_label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.testing == True:
            return (self.load_image(self.samples[idx]), self.samples[idx].replace(TEST_PATH + '/', ''))
        else:
            res = self.samples[idx]
            return (self.load_image(res[0]), res[1])

In [7]:
dataset = Imagedata(TRAIN_LABELS_PATH, TRAIN_PATH)

In [8]:
img_shape = dataset.__getitem__(0)[0].shape

In [77]:
test_dataset = Imagedata(TEST_LABELS_PATH, TEST_PATH, testing = True)

In [94]:
print(len(dataset) * SPLIT_FRAC)
print(len(dataset) * (1 - SPLIT_FRAC))

9074.6
36298.4


In [25]:
trainset, valset = random_split(dataset, [36298, 9075])
train_loader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(valset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [78]:
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [26]:
image_batch, text_batch = next(iter(train_loader))
print(image_batch.size(), text_batch)

torch.Size([32, 3, 224, 224]) tensor([13,  1,  1, 16,  8, 12, 16,  8, 23, 26,  4, 16, 26, 23, 21,  5, 17, 27,
        11,  0, 27,  8, 29,  7,  4,  4, 12, 15, 26, 28, 22, 10])


In [27]:
class MyNeuralNetwork(nn.Module):
    def __init__(self,  number_of_classes):
        super(MyNeuralNetwork,self).__init__()
        
        self.resnet = models.resnet50(pretrained =  True)
    #    self.resnet.fc=nn.Linear(self.resnet.fc.in_features, number_of_classes)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3)
        self.relu1=nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3)
        self.relu2=nn.ReLU()
        self.batchnorm2 = nn.BatchNorm2d(16)
        
        self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        self.relu3=nn.ReLU()
        self.batchnorm3 = nn.BatchNorm2d(32)
        
        self.conv4=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        self.relu4=nn.ReLU()
        self.batchnorm4 = nn.BatchNorm2d(64)
        
        self.conv5=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3)
        self.relu5=nn.ReLU()
        self.batchnorm5 = nn.BatchNorm2d(128)
            
        self.dropout = nn.Dropout(0.2)
        self.dropoutl = nn.Dropout(0.5)
    
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        self.fc = nn.Linear(1000, 256)
        self.fc1 = nn.Linear(256, number_of_classes)
        self.softmax = nn.Softmax(dim = 1)
    
    def forward(self, out):
       # out=self.conv1(out)
       # out=self.relu1(out)
       # out = self.batchnorm1(out)
        
       # out=self.conv2(out)
       # out=self.relu2(out)
       # out = self.maxpool2(out)

        #out = self.batchnorm2(out)
        
       # out = self.dropout(out)
        
        #out = self.conv3(out)
        #out = self.relu3(out)
        
       # out = self.batchnorm3(out)
        
       # out = self.conv4(out)
       # out = self.relu4(out)
        
       # out=self.maxpool2(out)
       
        #out = self.batchnorm4(out)
        
        #out = self.dropout(out)
        
       # out = self.conv5(out)
      #  out = self.relu5(out)
       
      #  out = self.batchnorm5(out)
      #  out = torch.flatten(out, start_dim = 1)
      #  out = self.fc(out)
       # out = self.dropoutl(out)
            
        out = self.resnet(out)
        out = self.fc(out)
        out = self.relu5(out)
        out = self.fc1(out)
        
        return out

In [28]:
model = MyNeuralNetwork(N_CLASSES)
model.to(DEVICE)

MyNeuralNetwork(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [16]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [23]:
from torchsummary import summary

summary(model, input_size = (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)


In [30]:
f1 = torchmetrics.F1Score(task="multiclass", num_classes=N_CLASSES).to(DEVICE)

def train_model(model, train_loader, val_loader, opt, n_epochs: int):
    train_loss = []
    val_loss = []
    f1_score = []
    
    for epoch in range(n_epochs):
        print(f"processing {epoch} epoch")
        ep_train_loss = []
        ep_val_loss = []
        ep_val_accuracy = []
        ep_f1_score = []
        
        model.train(True) 
        cnt = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE),y_batch.to(DEVICE)
            cnt+=1 
            if cnt % 100 == 0:
                print(f"{cnt} batches processed")
            
            predictions = model.forward(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            opt.step()
            opt.zero_grad()


            ep_train_loss.append(loss.item())
            #torch.cuda.empty_cache()

        model.eval() 
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                predictions = model(X_batch)
                loss = criterion( predictions, y_batch)
                f1_sc = f1(predictions, y_batch)
                ep_f1_score.append(f1_sc.item())
                ep_val_loss.append(loss.item())
              #  torch.cuda.empty_cache()
      #  torch.cuda.empty_cache()
        # print the results for this epoch:

        train_loss.append(np.mean(ep_train_loss))
        val_loss.append(np.mean(ep_val_loss))
        f1_score.append(np.mean(ep_f1_score))
        
        print(f"\t  training loss: {train_loss[-1]:.6f}")
        print(f"\tvalidation loss: {val_loss[-1]:.6f}")
        print(f"\t validation F score: {f1_score[-1]:.6f}")

    return train_loss , val_loss

In [31]:
train_loss, val_loss = train_model(model, train_loader, val_loader, optimizer, n_epochs=6)

processing 0 epoch
100 batches processed
200 batches processed
300 batches processed
400 batches processed
500 batches processed
600 batches processed
700 batches processed
800 batches processed
900 batches processed
1000 batches processed
1100 batches processed
	  training loss: 0.959411
	validation loss: 0.522687
	 validation F score: 0.858784
processing 1 epoch
100 batches processed
200 batches processed
300 batches processed
400 batches processed
500 batches processed
600 batches processed
700 batches processed
800 batches processed
900 batches processed
1000 batches processed
1100 batches processed
	  training loss: 0.507543
	validation loss: 0.462932
	 validation F score: 0.866996
processing 2 epoch
100 batches processed
200 batches processed
300 batches processed
400 batches processed
500 batches processed
600 batches processed
700 batches processed
800 batches processed
900 batches processed
1000 batches processed
1100 batches processed
	  training loss: 0.403607
	validation lo

In [85]:
test_ans = []
model.eval() 
with torch.no_grad():
    for X_batch, filename in test_loader:
        X_batch = X_batch.to(DEVICE)
        predictions = model(X_batch).cpu().numpy()
        res = np.argmax(predictions,axis = 1)
        for i in range(len(res)):
            test_ans.append((filename[i],res[i]))

In [87]:
test_ans = pd.DataFrame(test_ans, columns = ['image_id', 'label'])

In [88]:
test_ans['label'] = sport_enc.inverse_transform(test_ans['label'])

In [89]:
test_ans

,image_id,label
0,66f867a8-796c-4e3e-84ad-b55b60d0742d.jpeg,running
1,63c1f328-be56-4ada-b709-c14a1230189f.jpeg,horseback_riding
2,a3237adb-e5fe-4ea4-b258-588c06235319.jpeg,volleyball
3,396648f9-7486-44cb-aa3d-ae62627e414d.jpeg,taekwondo
4,a5a17d4b-69e8-42e4-9461-ff37cebc68e9.jpeg,pole_vault
...,...,...
19441,c971b54f-9f70-42fc-8fc2-daf7d5759a0a.jpeg,basketball
19442,e7b1df3b-40f4-4109-b64d-aa79bd5fe083.jpeg,gymnastics
19443,1f80841b-f17a-4d91-b627-78c5c82058be.jpeg,velo
19444,7af764f6-e252-4aac-a817-dbb99e34552e.jpeg,alpinism


In [92]:
test_ans.to_csv('final_final_submission.csv', index=False)